# Dunham's Data

## Setting up

In [86]:
from geopy.distance import geodesic
# import networkx as nx
import pandas as pd
# import seaborn as sns

In [87]:
# pd.set_option('display.max_rows', 999)

In [88]:
# sns.set()

In [89]:
# %matplotlib inline

## Loading data

In [90]:
col_used = [
    'DATE', 'CITY1', 'COUNTRY1', 'CITY2', 'COUNTRY2', 'WORKING', 'HOTEL_ADDRESS',
    'VENUE1', 'VENUE_TYPE1', 'VENUE2', 'VENUE_TYPE2', 'NOTES', 'SOURCE',
]
dunham_df = pd.read_excel('data/src/nureyev_EverydayItinerary_DatasetSample.xlsx',
                          usecols='A:G,I:N', names=col_used)
dunham_df.fillna('', inplace=True)

In [91]:
dunham_df.shape

(292, 13)

In [92]:
dunham_df.head()

,DATE,CITY1,COUNTRY1,CITY2,COUNTRY2,WORKING,HOTEL_ADDRESS,VENUE1,VENUE_TYPE1,VENUE2,VENUE_TYPE2,NOTES,SOURCE
0,1955-01-01 00:00:00,Moscow,Russia,,,y,,DEKADA - Bashkirian Decade of Literature & Art,,,,,
1,1955-08-24 00:00:00,Leningrad,Russia,,,y,,,,,,,
2,1956-01-01 00:00:00,Leningrad,Russia,,,y,,Kirov Theatre,,,,,
3,1956-02-01 00:00:00,Leningrad,Russia,,,y,,Kirov Theatre,,,,,
4,1956-03-01 00:00:00,Leningrad,Russia,,,y,,Kirov Theatre,,,,,


In [93]:
dunham_df.tail(1)

,DATE,CITY1,COUNTRY1,CITY2,COUNTRY2,WORKING,HOTEL_ADDRESS,VENUE1,VENUE_TYPE1,VENUE2,VENUE_TYPE2,NOTES,SOURCE
291,31/8/1992,Paris,France,,,y,,Palais Garnier,,,,,


## Pre-processing data

In [94]:
dunham_df.DATE = dunham_df.DATE.astype(str)
for col in dunham_df.columns[1:]:
    dunham_df[col] = dunham_df[col].apply(str.strip)

In [95]:
def clean_city(x, y):
    if x == 'Hamilton':
        if y == 'Canada':
            r = 'Hamilton (CA)'
        else: # y == 'New Zealand'
            r = 'Hamilton (NZ)'
#     elif x == 'in transit':
#         r = ''
    else:
        r = x
    return r

dunham_df.CITY1 = dunham_df.apply(lambda row: clean_city(row.CITY1, row.COUNTRY1), axis=1)
dunham_df.CITY2 = dunham_df.apply(lambda row: clean_city(row.CITY2, row.COUNTRY2), axis=1)

In [96]:
def clean_country(city, country):
    x = country
    y = city
    if y == 'Gibraltar':
        r = 'Gibraltar'
    elif x == 'Newfoundland':
        r = 'Canada'
    else:
        r = x
    return r

dunham_df.COUNTRY1 = dunham_df.apply(lambda row: clean_country(row.CITY1, row.COUNTRY1), axis=1)
dunham_df.COUNTRY2 = dunham_df.apply(lambda row: clean_country(row.CITY2, row.COUNTRY2), axis=1)

In [97]:
# def clean_working(x):
#     if x.startswith('[[') and x.endswith(']]'):
#         r = ''
#     else:
#         r = x
#     return r

# dunham_df.WORKING = dunham_df.WORKING.apply(clean_working)

In [98]:
def clean_hotel_address(x):
    r = x # do not clean for now
    return r

dunham_df.HOTEL_ADDRESS = dunham_df.HOTEL_ADDRESS.apply(clean_hotel_address)

In [99]:
# def clean_performance_yn(x):
#     if x == '?':
#         r = ''
#     elif x == 'N':
#         r = 'n'
#     else:
#         r = x
#     return r

# dunham_df.PERFORMANCE_YN = dunham_df.PERFORMANCE_YN.apply(clean_performance_yn)

In [100]:
# def clean_theater1(x):
#     r = x # do not clean for now
#     return r

# dunham_df.THEATER1 = dunham_df.THEATER1.apply(clean_theater1)

In [101]:
# def clean_theater2(x):
#     r = x # do not clean for now
#     return r

# dunham_df.THEATER2 = dunham_df.THEATER2.apply(clean_theater2)

In [102]:
# def clean_mode_transit(x):
#     if x == '[[chartered bus Reno to Los Angeles?]]':
#         r = 'bus'
#     elif x == 'drive':
#         r = 'car'
#     elif x == 'drive?':
#         r = 'car'
#     elif x.startswith('plane'):
#         r = 'plane'
#     elif x == 'train; boat train':
#         r = 'train, boat, train'
#     elif x == 'train; plane; boat':
#         r = 'train, plane, boat'
#     else:
#         r = x
#     return r

# dunham_df.MODE_TRANSIT = dunham_df.MODE_TRANSIT.apply(clean_mode_transit)

In [103]:
# def clean_transit_cities(x):
#     if x == 'Geneva; Rome':
#         r = 'Geneva (Switzerland), Rome (Italy)'
#     elif x == 'transit to Turin':
#         r = ''
#     elif x == 'transit through London':
#         r = 'London (England)'
#     elif x == 'transit: PanAm 201 flies NYC Laguardia - San Juan - Port of Spain - Belem':
#         r = 'San Juan (Puerto Rico), Port of Spain (Trinidad and Tobago), Belem (Brazil)'
#     elif x == 'Los Cerrillos airport':
#         r = ''
#     elif x == 'transit':
#         r = ''
#     elif x == 'destination':
#         r = ''
#     elif x == 'transit via Boston':
#         r = 'Boston (USA)'
#     elif x == 'in transit':
#         r = ''
#     elif x == 'last performance':
#         r = ''
#     elif x == 'London (Liverpool station), England; Harwich (Harwich Quay), England':
#         r = 'Harwich (England)'
#     elif x == 'New York':
#         r = ''
#     elif x == 'Chicago':
#         r = 'Chicago (USA)'
#     elif x == 'Gander, Newfoundland, Canada':
#         r = 'Gander (Canada)'
#     elif x == 'Zurich, Switzerland':
#         r = 'Zurich (Switzerland)'
#     elif x == 'Zurich':
#         r = 'Zurich (Switzerland)'
#     elif x == 'Rome, Italy; Geneva, Switzerland':
#         r = 'Rome (Italy), Geneva (Switzerland)'
#     elif x == 'Vevey, Switzerland':
#         r = 'Vevey (Switzerland)'
#     elif x == 'Amsterdam, Netherlands':
#         r = 'Amsterdam (Netherlands)'
#     elif x == 'Furet de P*?': # ???
#         r = ''
#     elif x == 'San Diego; Tijuana':
#         r = 'San Diego (USA), Tijuana (Mexico)'
#     elif x == 'Portworth':
#         r = 'Fort Worth (USA)'
#     elif x == 'Miami':
#         r = 'Miami (USA)'
#     elif x == 'Los Angeles':
#         r = 'Los Angeles (USA)'
#     elif x == "El Paso, TX----I think probably this was to go to Mexico, and I don't think she ever went to Mexico":
#         r = 'El Paso (USA)'
#     elif x == 'Honolulu':
#         r = 'Honolulu (Hawaii)'
#     elif x == 'Coolangatta':
#         r = 'Coolangatta (Australia)'
#     elif x == '[[Dunham Brisbane to Perth via Sydney and Melbourne--date open: https://osu.app.box.com/file/325139873147]]':
#         r = ''
#     elif x == 'Malacca':
#         r = 'Malacca (Malaysia)'
#     elif x == 'Amsterdam':
#         r = 'Amsterdam (Netherlands)'
#     elif x == 'Lyon; Bordeaux; Bayonne':
#         r = 'Lyons (France), Bordeaux (France), Bayonne (France)'
#     elif x == 'Irun, Spain':
#         r = 'Irun (Spain)'
#     elif x == 'Irun, Spain; Hendaye, France':
#         r = 'Irun (Spain), Hendaye (France)'
#     elif x == 'Marseilles':
#         r = 'Marseille (France)'
#     elif x == 'Piraeus, Greece':
#         r = 'Piraeus (Greece)'
#     elif x == 'Port Said, Egypt':
#         r = 'Port Said (Egypt)'
#     elif x == 'Munich':
#         r = 'Munich (Germany)'
#     elif x == 'Schwarzbach; Ravensberg':
#         r = 'Schwarzbach (Germany), Ravensberg (Germany)'
#     elif x == 'Colmar; Ligny en Barois; Rosay en Brie':
#         r = 'Colmar (France), Ligny-en-Barrois (France), Rozay-en-Brie (France)'
#     elif x == 'Verneson; Valence':
#         r = 'Vernaison (France), Valence (France)'
#     elif x == 'Alicante':
#         r = 'Alicante (Spain)'
#     elif x == 'Granada':
#         r = 'Granada (Spain)'
#     elif x == 'La Linea, Spain':
#         r = 'La Linea (Spain)'
#     else:
#         r = x
#     return r

# dunham_df.TRANSIT_CITIES = dunham_df.TRANSIT_CITIES.apply(clean_transit_cities)

In [104]:
def clean_notes(x):
    r = x # do not clean for now
    return r

dunham_df.NOTES = dunham_df.NOTES.apply(clean_notes)

In [105]:
def clean_source(x):
    r = x # do not clean for now
    return r

dunham_df.SOURCE = dunham_df.SOURCE.apply(clean_source)

In [106]:
dunham_df.fillna('', inplace=True)

## Processing data

In [107]:
geolocations = { # (lat, lon)
    'Mexico City': (19.433333, -99.133333),
    'New York City': (40.7127, -74.0059),
    'San Diego': (32.715, -117.1625),
    'San Francisco': (37.783333, -122.416667),
    'Stockton, CA': (37.975556, -121.300833),
    'Amsterdam' : ( 52.3730796,4.8924534),
    'Baltimore' : ( 39.2908816,-76.610759),
    'Bath' : ( 51.3813864,-2.3596963),
    'Berlin' : ( 52.510885,13.3989367),
    'Bologna' : ( 44.4938203,11.3426327),
    'Brussels' : ( 50.8465573,4.351697),
    'Budapest' : ( 47.4978789,19.0402383),
    'Cannes' : ( 43.5515198,7.0134418),
    'Chicago' : ( 41.8755616,-87.6244212),
    'Copenhagen' : ( 55.6867243,12.5700724),
    'Deauville' : ( 49.36,0.0752778),
    'Dublin' : ( 53.3493795,-6.2605593),
    'Florence' : ( 43.7697955,11.2556404),
    'Frankfurt' : ( 50.1106444,8.6820917),
    'Genoa' : ( 44.40726,8.9338624),
    'Hamburg' : ( 53.550341,10.000654),
    'Holmdel' : ( 40.3451095,-74.1840322),
    'jerusalem' : ( 31.7788242,35.2257626),
    'Leningrad' : ( 59.938732,30.316229),
    'Liverpool' : ( 53.4071991,-2.99168),
    'London' : ( 51.5074456,-0.1277653),
    'Madrid' : ( 40.4167047,-3.7035825),
    'Mantua' : ( 45.1692628,10.67083652),
    'Marseille' : ( 43.2961743,5.3699525),
    'Melbourne' : ( -37.8142454,144.9631732),
    'Mexico City' : ( 19.4326296,-99.1331785),
    'Milano' : ( 45.4641943,9.1896346),
    'Monte Carlo' : ( 43.7402961,7.426559),
    'Montreal' : ( 45.5031824,-73.5698065),
    'Moscow' : ( 55.7505412,37.6174782),
    'Nervi' : ( 44.3833121,9.0391295),
    'New York' : ( 40.7127281,-74.0060152),
    'North Russia' : ( 40.2347473,-84.4071014),
    'Palma de Mallorca' : ( 39.5695818,2.6500745),
    'Paris' : ( 48.8588897,2.320041022),
    'Rome' : ( 41.8933203,12.4829321),
    'Scheveningen' : ( 52.1067449,4.2736937),
    'Spoleto' : ( 52.663642,-1.580326),
    'Stockholm' : ( 59.3251172,18.0710935),
    'Stuttgart' : ( 48.7784485,9.1800132),
    'Toronto' : ( 43.6534817,-79.3839347),
    'Turin' : ( 45.0677551,7.6824892),
    'Utrecht' : ( 52.0809856,5.127683969),
    'Venice' : ( 45.4371908,12.3345898),
    'Verona' : ( 45.4424977,10.98573769),
    'Vienna' : ( 48.2083537,16.3725042), 
}

### Grouping locations

In [108]:
distances = set()
cities = set(dunham_df.CITY1.unique()).union(dunham_df.CITY2.unique())
cities.discard('')
cities.discard('in transit')
for i, city1 in enumerate(cities):
    p1 = geolocations[city1]
    for j, city2 in enumerate(cities):
        if j > i: # and city1 != city2:
            p2 = geolocations[city2]
            dst = geodesic(p1, p2).km
            distances.add((city1, city2, dst))

In [109]:
len(distances)

1035

In [110]:
sorted(distances, key=lambda x: x[2])[:10]

[('Genoa', 'Nervi', 8.798867936283308),
 ('Amsterdam', 'Utrecht', 36.258983623548914),
 ('Verona', 'Mantua', 39.14139508892541),
 ('Cannes', 'Monte Carlo', 39.380948897756255),
 ('Holmdel', 'New York', 43.51965193523631),
 ('Amsterdam', 'Scheveningen', 51.62160815324791),
 ('Utrecht', 'Scheveningen', 58.59740926438325),
 ('Bologna', 'Florence', 80.75053306878519),
 ('Bologna', 'Mantua', 91.95733272938759),
 ('Verona', 'Venice', 105.53628450707883)]

In [111]:
THRESHOLD = 15

In [112]:
grouped_cities_dct = {
    'Petion-Ville': 'Port-au-Prince',
    'Los Angeles': 'Los Angeles area',
    'Hollywood': 'Los Angeles area',
    'Beverly Hills': 'Los Angeles area',
    'Fiesole': 'Florence',
#     'Mosman': 'Sydney',
}

In [113]:
for k in grouped_cities_dct:
    dunham_df.replace(k, grouped_cities_dct[k], inplace=True)

In [114]:
dunham_df.shape

(292, 13)

## Saving data

In [115]:
dunham_df.to_csv('data/out/nureyev_1955-92.csv', index=False)